# Step-by-Step Image Feature Extraction
Run each cell below one by one to extract visual features from ad creatives.

In [ ]:
# 📦 Import necessary libraries
import cv2
import os
import pandas as pd
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt

In [ ]:
# 📁 Load the ad metadata CSV
csv_path = "ads_data.csv"  # Make sure this file is in your working directory
ads_df = pd.read_csv(csv_path)
ads_df.head()

In [ ]:
# 🗂️ Update image paths if your images are inside a subfolder (like 'images/data/')
ads_df["image_path"] = ads_df["image_filename"].apply(lambda x: os.path.join("images/data", x))
ads_df.head()

## 🧠 Feature Extraction Functions

In [ ]:
def calculate_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return np.mean(hsv[:, :, 2])

def calculate_contrast(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray.std()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return len(faces) > 0

def get_dominant_color(image, k=4):
    data = image.reshape((-1, 3))
    data = np.float32(data)
    _, labels, centers = cv2.kmeans(data, k, None,
                                    (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0),
                                    10, cv2.KMEANS_RANDOM_CENTERS)
    counts = Counter(labels.flatten())
    dominant = centers[np.argmax(list(counts.values()))]
    return tuple(map(int, dominant))

## 🧪 Extract Features in Batches (Modify range as needed)

In [ ]:
# Example: Run for first 20 images. Change range for batches.
features = []
for path in ads_df["image_path"].iloc[0:20]:  # Adjust range
    try:
        image = cv2.imread(path)
        if image is None:
            raise ValueError("Image not loaded.")
        brightness = calculate_brightness(image)
        contrast = calculate_contrast(image)
        face_present = detect_faces(image)
        dominant_color = get_dominant_color(image)
        features.append({
            "image_path": path,
            "brightness": brightness,
            "contrast": contrast,
            "face_detected": face_present,
            "dominant_color": dominant_color
        })
    except Exception:
        features.append({
            "image_path": path,
            "brightness": None,
            "contrast": None,
            "face_detected": None,
            "dominant_color": None
        })

features_df = pd.DataFrame(features)
features_df.head()

In [ ]:
# 💾 Save your features to a CSV file
features_df.to_csv("image_features_extracted.csv", index=False)